In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import html
from IPython.display import HTML

print('Libraries imported.')

Libraries imported.


In [120]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


In [57]:
apartment_df = pd.read_csv("apartment_dataframe.csv")


In [58]:
apartment_df.drop(columns =["Unnamed: 0"])

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,'Clothing Store','College Gym','College Gym'],'College Rec Center','Convenience Store','Convenience Store'],'Department Store','Department Store'],'Discount Store','Electronics Store','Furniture / Home Store','Furniture / Home Store'],'General Travel','Gym / Fitness Center','Hospital','IT Services','Miscellaneous Shop','Miscellaneous Shop'],'Mobile Phone Shop','None','None'],'Nudist Beach','Paper / Office Supplies Store','Paper / Office Supplies Store'],'Smoke Shop','Smoke Shop'],'Tourist Information Center','Veterinarian',[''],['Convenience Store',['Department Store',['Furniture / Home Store',['General Travel',['Gym / Fitness Center',['Miscellaneous Shop',['None',['Smoke Shop',['Tourist Information Center'
0,"171, Priestman Street, Rabbit Town, Uptown, Fr...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","[""['Smoke Shop'"", "" 'Discount Store'"", "" 'Misc...",450.0,0,1,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,3,1,0,4,0,0,0,0,0,0,0,0,1,0
1,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","[""['Smoke Shop'"", "" 'Discount Store'"", "" 'Misc...",500.0,0,1,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,3,1,0,4,0,0,0,0,0,0,0,0,1,0
2,"George Street, Carleton, Downtown, Fredericton...","['The UPS Store', 'UPS Store', 'Kwik-Way Conve...","[""['Miscellaneous Shop'"", "" 'Paper / Office Su...",550.0,2,1,5,0,0,0,5,0,0,0,0,0,4,1,5,4,0,0,4,0,0,0,0,4,5,0,0,0,5,4,0,0,0,0,0,0,1,0,0,0
3,"George Street, Carleton, Downtown, Fredericton...","['The UPS Store', 'UPS Store', 'Kwik-Way Conve...","[""['Miscellaneous Shop'"", "" 'Paper / Office Su...",600.0,0,1,5,0,0,0,5,0,0,0,0,0,4,1,5,4,0,0,4,0,0,0,0,4,5,0,0,0,5,4,0,0,0,0,0,0,1,0,0,0
4,"York Street, Rabbit Town, Downtown, Fredericto...","['The UPS Store', 'UPS Store', 'Kwik-Way Conve...","[""['Miscellaneous Shop'"", "" 'Paper / Office Su...",650.0,0,1,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,5,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0
5,"Canterbury Drive, Dunn's Crossing, Skyline Acr...","[""Raymonde's General Store"", ""Raymonde's Gener...","[""['None'"", "" 'None'"", "" 'None'"", "" 'None'"", ""...",700.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,"31, Corbett Avenue, Station Pointe, Barker's P...",[],"[""['']""]",750.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","[""['Miscellaneous Shop'"", "" 'Gym / Fitness Cen...",750.0,1,1,0,8,0,4,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","[""['Miscellaneous Shop'"", "" 'Gym / Fitness Cen...",750.0,1,1,0,8,0,4,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,"King Street, Victoria Circle, Downtown, Freder...","['Urban Almanac', 'Urban Almanac', 'Urban Alma...","[""['Furniture / Home Store'"", "" 'Furniture / H...",780.0,1,1,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [59]:
#To add The neighborhoods for each address on the dataframe:

address_list = apartment_df['Apartment Address'].to_list()
#address_list

In [60]:
neighborhoods = []
for i in range(len(address_list)):
    
    splited_add = address_list[i].split(",")
    if len(splited_add) > 8:
        neighborhoods.append(splited_add[-7])
    else:
        neighborhoods.append(splited_add[1])
#len(neighborhoods)
    

In [61]:
#Organizing the DataFrame:
apartment_df["Neighborhood"] = neighborhoods
#apartment_df = apartment_df[["Neighborhood", "Apartment Address","Amenities", "Amenity type", "Rent price", "Bedrooms","Bathrooms", "Clothing Store", "College Gym", "College Rec Center", "Convenience Store", "Department Store", "Discount Store", "Furniture / Home Store", "General Travel", "Gym / Fitness Center", "Miscellaneous Shop", "Paper / Office Supplies Store", "Smoke Shop","Tourist Information Center", "Veterinarian"]]
#apartment_df["Rent price"] = apartment_df["Rent price"].astype("float")
#apartment_df.dtypes

In [62]:

apartment_df['Unnamed: 0'] = apartment_df['Neighborhood'].values

In [63]:
apartment_df = apartment_df.drop('Neighborhood', axis = 1)


In [65]:
apartment_df = apartment_df.rename(columns = {"Unnamed: 0": 'Neighborhood'})

In [66]:
apartment_df

,Neighborhood,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,'Clothing Store','College Gym','College Gym'],'College Rec Center','Convenience Store','Convenience Store'],'Department Store','Department Store'],'Discount Store','Electronics Store','Furniture / Home Store','Furniture / Home Store'],'General Travel','Gym / Fitness Center','Hospital','IT Services','Miscellaneous Shop','Miscellaneous Shop'],'Mobile Phone Shop','None','None'],'Nudist Beach','Paper / Office Supplies Store','Paper / Office Supplies Store'],'Smoke Shop','Smoke Shop'],'Tourist Information Center','Veterinarian',[''],['Convenience Store',['Department Store',['Furniture / Home Store',['General Travel',['Gym / Fitness Center',['Miscellaneous Shop',['None',['Smoke Shop',['Tourist Information Center'
0,Rabbit Town,"171, Priestman Street, Rabbit Town, Uptown, Fr...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","[""['Smoke Shop'"", "" 'Discount Store'"", "" 'Misc...",450.0,0,1,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,3,1,0,4,0,0,0,0,0,0,0,0,1,0
1,Rabbit Town,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","[""['Smoke Shop'"", "" 'Discount Store'"", "" 'Misc...",500.0,0,1,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,3,1,0,4,0,0,0,0,0,0,0,0,1,0
2,Carleton,"George Street, Carleton, Downtown, Fredericton...","['The UPS Store', 'UPS Store', 'Kwik-Way Conve...","[""['Miscellaneous Shop'"", "" 'Paper / Office Su...",550.0,2,1,5,0,0,0,5,0,0,0,0,0,4,1,5,4,0,0,4,0,0,0,0,4,5,0,0,0,5,4,0,0,0,0,0,0,1,0,0,0
3,Carleton,"George Street, Carleton, Downtown, Fredericton...","['The UPS Store', 'UPS Store', 'Kwik-Way Conve...","[""['Miscellaneous Shop'"", "" 'Paper / Office Su...",600.0,0,1,5,0,0,0,5,0,0,0,0,0,4,1,5,4,0,0,4,0,0,0,0,4,5,0,0,0,5,4,0,0,0,0,0,0,1,0,0,0
4,Rabbit Town,"York Street, Rabbit Town, Downtown, Fredericto...","['The UPS Store', 'UPS Store', 'Kwik-Way Conve...","[""['Miscellaneous Shop'"", "" 'Paper / Office Su...",650.0,0,1,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,5,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0
5,Dunn's Crossing,"Canterbury Drive, Dunn's Crossing, Skyline Acr...","[""Raymonde's General Store"", ""Raymonde's Gener...","[""['None'"", "" 'None'"", "" 'None'"", "" 'None'"", ""...",700.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,Station Pointe,"31, Corbett Avenue, Station Pointe, Barker's P...",[],"[""['']""]",750.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,College Hill,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","[""['Miscellaneous Shop'"", "" 'Gym / Fitness Cen...",750.0,1,1,0,8,0,4,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,College Hill,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","[""['Miscellaneous Shop'"", "" 'Gym / Fitness Cen...",750.0,1,1,0,8,0,4,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,Victoria Circle,"King Street, Victoria Circle, Downtown, Freder...","['Urban Almanac', 'Urban Almanac', 'Urban Alma...","[""['Furniture / Home Store'"", "" 'Furniture / H...",780.0,1,1,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [69]:
grouped_df = apartment_df.groupby(["Neighborhood"]).mean().reset_index()
grouped_df = grouped_df.drop([0])
grouped_df

,Neighborhood,Rent price,Bedrooms,Bathrooms,'Clothing Store','College Gym','College Gym'],'College Rec Center','Convenience Store','Convenience Store'],'Department Store','Department Store'],'Discount Store','Electronics Store','Furniture / Home Store','Furniture / Home Store'],'General Travel','Gym / Fitness Center','Hospital','IT Services','Miscellaneous Shop','Miscellaneous Shop'],'Mobile Phone Shop','None','None'],'Nudist Beach','Paper / Office Supplies Store','Paper / Office Supplies Store'],'Smoke Shop','Smoke Shop'],'Tourist Information Center','Veterinarian',[''],['Convenience Store',['Department Store',['Furniture / Home Store',['General Travel',['Gym / Fitness Center',['Miscellaneous Shop',['None',['Smoke Shop',['Tourist Information Center'
1,Carleton,575.000000,1.000000,1.000000,5.000000,0.000000,0.0,0.0,5.000000,0.000000,0.000000,0.0,0.000000,0.000000,4.0,1.0,5.0,4.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.0,0.000000,4.000000,5.000000,0.000000,0.0,0.000000,5.0,4.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0
2,College Hill,914.166667,1.500000,1.000000,0.833333,5.333333,0.0,2.0,0.000000,0.000000,0.833333,0.0,0.833333,1.666667,0.0,0.0,0.0,2.666667,1.333333,0.833333,4.833333,0.333333,0.833333,0.0,0.000000,0.666667,1.333333,0.333333,1.0,0.333333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.666667,0.000000,0.333333,0.0
3,Dunn's Crossing,914.166667,1.666667,1.166667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.666667,0.000000,0.0,0.0,0.0,0.0,0.000000,0.333333,0.000000,0.0
4,Garden Creek,995.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
5,NB Power,925.000000,1.500000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
6,Prospect Street,1185.000000,2.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
7,Queen Street,979.000000,1.500000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,4.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,4.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0
8,Rabbit Town,750.000000,1.333333,1.000000,0.000000,0.000000,0.0,0.0,1.222222,0.333333,3.333333,0.0,2.222222,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.222222,0.000000,0.000000,0.0,0.000000,0.444444,1.111111,0.000000,2.0,0.666667,0.0,3.555556,0.000000,0.111111,0.0,0.0,0.0,0.0,0.222222,0.000000,0.666667,0.0
9,Station Pointe,750.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
10,Town Platt,950.000000,2.000000,1.000000,0.000000,7.000000,1.0,4.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0
